# StreetComplete Statistics

Analysis of OpenStreetMap contributions using StreetComplete, a mobile mapping application.

In [1]:
import duckdb
import util

util.init()

## Monthly Total and Percent of Edits and Contributors Using StreetComplete

In [2]:
# Monthly StreetComplete statistics with totals and percentages
df_monthly = duckdb.sql("""
WITH monthly_total AS (
    SELECT 
        year,
        month,
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        COUNT(DISTINCT user_name) as total_contributors,
        CAST(SUM(edit_count) as BIGINT) as total_edits
    FROM '../changeset_data/year=*/month=*/*.parquet'
    GROUP BY year, month
),
monthly_streetcomplete AS (
    SELECT 
        year,
        month,
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        COUNT(DISTINCT user_name) as sc_contributors,
        CAST(SUM(edit_count) as BIGINT) as sc_edits
    FROM '../changeset_data/year=*/month=*/*.parquet'
    WHERE created_by = 'StreetComplete'
    GROUP BY year, month
)
SELECT 
    mt.months,
    mt.total_contributors as "Total Contributors",
    COALESCE(msc.sc_contributors, 0) as "StreetComplete Contributors",
    mt.total_edits as "Total Edits",
    COALESCE(msc.sc_edits, 0) as "StreetComplete Edits",
    ROUND((COALESCE(msc.sc_contributors, 0) * 100.0) / mt.total_contributors, 2) as "Percent Contributors using StreetComplete",
    ROUND((COALESCE(msc.sc_edits, 0) * 100.0) / mt.total_edits, 2) as "Percent Edits from StreetComplete"
FROM monthly_total mt
LEFT JOIN monthly_streetcomplete msc ON mt.year = msc.year AND mt.month = msc.month
ORDER BY mt.year, mt.month
""").df()

util.show_figure(
    [
        util.FigureConfig(
            title="Monthly StreetComplete Edits",
            label="Total Edits",
            x_col="months",
            y_col="StreetComplete Edits",
            query_or_df=df_monthly,
        ),
        util.FigureConfig(
            title="Percentage of Edits from StreetComplete",
            label="Edits Percentage",
            x_col="months",
            y_col="Percent Edits from StreetComplete",
            y_unit_hover_template="%",
            query_or_df=df_monthly,
        ),
        util.FigureConfig(
            title="Monthly StreetComplete Contributors",
            label="Total Contributors",
            x_col="months",
            y_col="StreetComplete Contributors",
            query_or_df=df_monthly,
        ),
        util.FigureConfig(
            title="Percentage of Contributors using StreetComplete",
            label="Contributors Percentage",
            x_col="months",
            y_col="Percent Contributors using StreetComplete",
            y_unit_hover_template="%",
            query_or_df=df_monthly,
        ),
    ]
)

## Yearly Edits and Contributors for Each StreetComplete Quest

In [3]:
# Yearly breakdown by StreetComplete quest types
query_yearly_quests = """
WITH user_first_year AS (
    SELECT 
        user_name,
        streetcomplete_quest,
        MIN(year) as first_year
    FROM '../changeset_data/year=*/month=*/*.parquet'
    WHERE streetcomplete_quest IS NOT NULL AND created_by = 'StreetComplete'
    GROUP BY user_name, streetcomplete_quest
),
quest_totals AS (
    SELECT
        streetcomplete_quest as "StreetComplete Quest",
        CAST(SUM(edit_count) as BIGINT) as total_edits_all_time,
        CAST(SUM(CASE WHEN year >= 2021 THEN edit_count ELSE 0 END) as BIGINT) as total_edits_2021_now,
        CAST(COUNT(DISTINCT user_name) as BIGINT) as total_contributors_all_time,
        CAST(COUNT(DISTINCT CASE WHEN year >= 2021 THEN user_name END) as BIGINT) as total_contributors_2021_now
    FROM '../changeset_data/year=*/month=*/*.parquet'
    WHERE streetcomplete_quest IS NOT NULL AND created_by = 'StreetComplete'
    GROUP BY streetcomplete_quest
),
yearly_metrics AS (
    SELECT
        d.year,
        d.streetcomplete_quest as "StreetComplete Quest",
        CAST(SUM(d.edit_count) as BIGINT) as "Edits",
        CAST(COUNT(DISTINCT d.user_name) as BIGINT) as "Contributors",
        CAST(COUNT(DISTINCT CASE WHEN ufy.first_year = d.year THEN d.user_name END) as BIGINT) as "New Contributors"
    FROM '../changeset_data/year=*/month=*/*.parquet' d
    LEFT JOIN user_first_year ufy 
        ON d.user_name = ufy.user_name AND d.streetcomplete_quest = ufy.streetcomplete_quest
    WHERE d.streetcomplete_quest IS NOT NULL  AND created_by = 'StreetComplete'
    GROUP BY d.year, d.streetcomplete_quest
)
SELECT 
    ym.year,
    ym."StreetComplete Quest",
    ym."Edits",
    ym."New Contributors",
    ym."Contributors",
    qt.total_edits_all_time as "Total Edits",
    qt.total_edits_2021_now as "Total Edits (2021 - Now)",
    qt.total_contributors_all_time as "Total Contributors",
    qt.total_contributors_2021_now as "Total Contributors (2021 - Now)"
FROM yearly_metrics ym
JOIN quest_totals qt
    ON ym."StreetComplete Quest" = qt."StreetComplete Quest"
ORDER BY year DESC, "Edits" DESC
"""

df_yearly_quests = duckdb.sql(query_yearly_quests).df()

# Get all StreetComplete quests
all_contributors = df_yearly_quests.groupby("StreetComplete Quest")["Total Contributors"].first()
all_contributors_2021_now = df_yearly_quests.groupby("StreetComplete Quest")["Total Contributors (2021 - Now)"].first()
all_edits = df_yearly_quests.groupby("StreetComplete Quest")["Total Edits"].first()
all_edits_2021_now = df_yearly_quests.groupby("StreetComplete Quest")["Total Edits (2021 - Now)"].first()

table_configs = [
    util.TableConfig(
        title="All StreetComplete Quests by Edits (All Time)",
        query_or_df=df_yearly_quests[df_yearly_quests["StreetComplete Quest"].isin(all_edits.index)],
        x_axis_col="year",
        y_axis_col="StreetComplete Quest",
        value_col="Edits",
        center_columns=["Rank", "StreetComplete Quest"],
        sum_col="Total Edits",
    ),
    util.TableConfig(
        title="All StreetComplete Quests by Edits (2021 - Now)",
        query_or_df=df_yearly_quests[
            (df_yearly_quests["StreetComplete Quest"].isin(all_edits_2021_now.index))
            & (df_yearly_quests["year"] >= 2021)
        ],
        x_axis_col="year",
        y_axis_col="StreetComplete Quest",
        value_col="Edits",
        center_columns=["Rank", "StreetComplete Quest"],
        sum_col="Total Edits (2021 - Now)",
    ),
    util.TableConfig(
        title="All StreetComplete Quests by Contributors (All Time)",
        query_or_df=df_yearly_quests[df_yearly_quests["StreetComplete Quest"].isin(all_contributors.index)],
        x_axis_col="year",
        y_axis_col="StreetComplete Quest",
        value_col="Contributors",
        center_columns=["Rank", "StreetComplete Quest"],
        sum_col="Total Contributors",
    ),
    util.TableConfig(
        title="All StreetComplete Quests by Contributors (2021 - Now)",
        query_or_df=df_yearly_quests[
            (df_yearly_quests["StreetComplete Quest"].isin(all_contributors_2021_now.index))
            & (df_yearly_quests["year"] >= 2021)
        ],
        x_axis_col="year",
        y_axis_col="StreetComplete Quest",
        value_col="Contributors",
        center_columns=["Rank", "StreetComplete Quest"],
        sum_col="Total Contributors (2021 - Now)",
    ),
]

util.show_tables(table_configs)

Rank,StreetComplete Quest,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,Total Edits
1,AddRoadSurface,0,"699,689","534,690","428,070","495,755","1,268,801","1,372,576","1,295,203","1,097,473","1,004,706","62,298","8,259,261"
2,AddBuildingType,0,0,"216,587","277,087","629,114","1,734,569","1,569,203","1,844,920","424,202","200,073","10,310","6,906,065"
3,AddPathSurface,0,0,"42,597","147,386","305,052","945,990","792,385","770,873","696,578","718,129","45,234","4,464,224"
4,AddBuildingLevels,0,"84,190","137,679","159,553","396,895","1,191,453","925,245","666,977","367,324","255,946","13,593","4,198,855"
5,AddWayLit,0,"108,264","226,736","300,578","484,734","1,254,731","830,955","268,971","181,533","196,161","9,524","3,862,187"
6,AddRoofShape,0,"20,325","53,298","64,678","175,129","859,815","479,642","514,202","300,123","199,564","10,014","2,676,790"
7,AddLanes,0,30,4,0,"86,518","360,137","325,064","462,378","620,850","523,541","11,413","2,389,935"
8,WayLitOverlay,0,0,0,0,0,0,"297,388","564,698","701,792","505,429","53,580","2,122,887"
9,AddHousenumber,0,"22,274","41,388","46,692","140,891","449,951","373,106","363,683","274,675","265,884","15,212","1,993,756"
10,AddTactilePavingCrosswalk,0,"24,245","32,007","40,078","69,991","247,461","255,770","287,829","301,583","273,678","18,370","1,551,012"


## Accumulated Edits and Contributors of Top 10 StreetComplete Quests

In [4]:
# Top 10 StreetComplete quests with monthly trends
df_top_quests = duckdb.sql("""
WITH top_quests AS (
    SELECT streetcomplete_quest
    FROM (
        SELECT
            streetcomplete_quest,
            COUNT(DISTINCT user_name) as total_contributors
        FROM '../changeset_data/year=*/month=*/*.parquet'
        WHERE created_by = 'StreetComplete'
        GROUP BY streetcomplete_quest
        ORDER BY total_contributors DESC
        LIMIT 10
    )
),
monthly_quest_data AS (
    SELECT 
        year,
        month,
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        streetcomplete_quest,
        COUNT(DISTINCT user_name) as "Contributors",
        SUM(edit_count) as "Edits"
    FROM '../changeset_data/year=*/month=*/*.parquet'
    WHERE streetcomplete_quest IN (SELECT streetcomplete_quest FROM top_quests)
    GROUP BY year, month, streetcomplete_quest
)
SELECT 
    months,
    streetcomplete_quest,
    "Contributors",
    "Edits",
    SUM("Contributors") OVER (PARTITION BY streetcomplete_quest ORDER BY year, month) as "Accumulated Contributors",
    SUM("Edits") OVER (PARTITION BY streetcomplete_quest ORDER BY year, month) as "Accumulated Edits"
FROM monthly_quest_data
ORDER BY year, month, streetcomplete_quest
""").df()

util.show_figure(
    [
        util.FigureConfig(
            title="Monthly Contributors by Top 10 StreetComplete Quests",
            label="Monthly Contributors",
            x_col="months",
            y_col="Contributors",
            group_col="streetcomplete_quest",
            query_or_df=df_top_quests,
        ),
        util.FigureConfig(
            title="Monthly Edits by Top 10 StreetComplete Quests",
            label="Monthly Edits",
            x_col="months",
            y_col="Edits",
            group_col="streetcomplete_quest",
            query_or_df=df_top_quests,
        ),
        util.FigureConfig(
            title="Accumulated Contributors by Top 10 StreetComplete Quests",
            label="Accumulated Contributors",
            x_col="months",
            y_col="Accumulated Contributors",
            group_col="streetcomplete_quest",
            query_or_df=df_top_quests,
        ),
        util.FigureConfig(
            title="Accumulated Edits by Top 10 StreetComplete Quests",
            label="Accumulated Edits",
            x_col="months",
            y_col="Accumulated Edits",
            group_col="streetcomplete_quest",
            query_or_df=df_top_quests,
        ),
    ]
)

## Map with Total Edits Using StreetComplete

In [5]:
df_map = duckdb.sql("""
SELECT
    mid_pos_x as x,
    mid_pos_y as y,
    SUM(edit_count) as z
FROM '../changeset_data/year=*/month=*/*.parquet'
WHERE created_by = 'StreetComplete'
    AND mid_pos_x IS NOT NULL 
    AND mid_pos_y IS NOT NULL
GROUP BY mid_pos_x, mid_pos_y
""").df()

util.show_figure(
    [
        util.FigureConfig(
            title="Global Distribution of StreetComplete Edits",
            x_col="x",
            y_col="y",
            z_col="z",
            query_or_df=df_map,
            plot_type="map",
        )
    ]
)